In [3]:
# Código para aplicar la técnica Vocal Tract Length Normalization (VTLN) a los audios originales

# Instalar las dependencias
!pip install librosa soundfile

# Importar librerías
import librosa
import numpy as np
import os
import soundfile as sf  # Usar soundfile para manejar archivos de audio
from google.colab import drive

drive.mount('/content/drive')

# Configuración de rutas
input_folder = '/content/drive/MyDrive/Data_Augmentation/JUAN/AUDIO_DATA_AUGM/DATA'  # Carpeta con audios originales
output_folder = '/content/drive/MyDrive/Data_Augmentation/JUAN/AUDIO_DATA_AUGM/RESULTS/SpectrogramAugmentation/VTLN/AUDIOS'  # Carpeta para guardar los audios modificados
os.makedirs(output_folder, exist_ok=True)

# Función para aplicar Vocal Tract Length Normalization (VTLN)
def apply_vtln(audio, sr, alpha):
    # Obtener espectrograma
    spectrogram = librosa.stft(audio)
    magnitudes, phases = np.abs(spectrogram), np.angle(spectrogram)

    # Crear una transformación warp-lineal para las frecuencias
    n_bins = magnitudes.shape[0]
    original_frequencies = np.linspace(0, sr // 2, n_bins)
    warped_frequencies = np.clip(original_frequencies * alpha, 0, sr // 2)

    # Interpolar las magnitudes para aplicar el warp
    warped_magnitudes = np.zeros_like(magnitudes)
    for i in range(magnitudes.shape[1]):  # Iterar sobre las columnas (tiempo)
        warped_magnitudes[:, i] = np.interp(original_frequencies, warped_frequencies, magnitudes[:, i])

    # Reconstruir espectrograma
    warped_spectrogram = warped_magnitudes * np.exp(1j * phases)
    warped_audio = librosa.istft(warped_spectrogram)
    return warped_audio

# Procesar los archivos
for file_name in os.listdir(input_folder):
    if file_name.endswith('.wav'):
        print(f"Procesando: {file_name}")
        file_path = os.path.join(input_folder, file_name)

        # Generar un valor aleatorio para alpha
        alpha = round(np.random.uniform(0.9, 1.1), 2)  # Valor de alpha entre 0.9 y 1.1
        print(f"  -> Alpha aplicado: {alpha}")

        # Cargar el audio
        audio, sr = librosa.load(file_path, sr=None)

        # Aplicar la técnica VTLN
        processed_audio = apply_vtln(audio, sr, alpha)

        # Guardar el archivo modificado
        output_path = os.path.join(output_folder, f"vtln_alpha{alpha}_{file_name}")
        sf.write(output_path, processed_audio, sr)  # Guardar usando soundfile

print("Archivos procesados con Vocal Tract Length Normalization (VTLN).")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Procesando: VNE_90_P2_17_VM.wav
  -> Alpha aplicado: 1.1
Procesando: VNE_103_P1_88_MH.wav
  -> Alpha aplicado: 1.09
Procesando: VNE_92_P2_18_SI.wav
  -> Alpha aplicado: 1.09
Procesando: VNE_102_P2_79_TM.wav
  -> Alpha aplicado: 0.95
Procesando: VNE_31_P2_42_BI.wav
  -> Alpha aplicado: 1.0
Procesando: VNE_104_P1_57_GE.wav
  -> Alpha aplicado: 0.95
Archivos procesados con Vocal Tract Length Normalization (VTLN).
